# Midterm Question 1
Analysis of Enron Email dataset

In [ ]:
import tarfile
import numpy as np 

tar = tarfile.open("/Users/mahesh/midterm/data/enron/enron_mail_20150507.tgz")
for member in tar.getmembers():
    f = tar.extractfile(member)
    if f is not None:
            content = f.read()
            #print (content.count('\n'))
            print(content)
            
            #print ("%s has %d spaces" + str(member,content.count(" ")))
            #print ("%s has %d characters" +str(member, len(content)))
            #sys.exit()
tar.close()

In [ ]:
import tarfile
import numpy as np 

tar = tarfile.open("/Users/mahesh/midterm/data/enron/enron_mail_20150507.tgz")
for member in tar.getmembers():
    f = tar.extractfile(member)
    if f is not None:
            content = f.read()
            #print (content.count('\n'))
            #print(content)
            print(type(content))
            
            #print ("%s has %d spaces" + str(member,content.count(" ")))
            #print ("%s has %d characters" +str(member, len(content)))
            #sys.exit()
tar.close()

In [13]:
#Opening emails

from email.parser import Parser

file_to_read = "C:\\Users\\mahesh\\midterm\\data\\enron\\enron_mail_20150507\\maildir\\lay-k\\all_documents\\1"

with open(file_to_read, "r") as f:
    data = f.read()  

#print(data)

email = Parser().parsestr(data)#Creating email Parser instance
print(type(email))
print("\nTo: " , email['to']) 
print("\n From: " , email['from']) 

print("\n Subject: " , email['subject']) 

print("\n \n Body: " , email.get_payload())

<class 'email.message.Message'>

To:  mmilken@knowledgeu.com

 From:  rosalee.fleming@enron.com

 Subject:  Re: testing

 
 Body:  Hi -

We did receive the e-mail.

Rosalee for Ken Lay





"Michael Milken" <mmilken@knowledgeu.com> on 07/02/99 10:21:40 AM
To: Kenneth Lay/Corp/Enron@Enron
cc:  
Subject: testing









# Analysis 1:Finding who sent the most  messages and who received the most
# In the To list, the top person is Richard Shapiro.
# The highest From field, which is emails received from, is Kay Mann.

In [45]:
import os
from collections import Counter

from email.parser import Parser
rootdir = "C:\\Users\\mahesh\\midterm\\data\\enron\\enron_mail_20150507\\maildir"


def email_analyse(infile, to_email_list, from_email_list, email_body):
    with open(infile, "r") as f:
        data = f.read() 

    email = Parser().parsestr(data)
    
    if email['to']:
        email_to = email['to']
        email_to = email_to.replace("\n", "")
        email_to = email_to.replace("\t", "")
        email_to = email_to.replace(" ", "")

        email_to = email_to.split(",")

        for email_to_1 in email_to:
            to_email_list.append(email_to_1)

    from_email_list.append(email['from'])


to_email_list = []
from_email_list = []
email_body = []

for directory, subdirectory, filenames in  os.walk(rootdir):
    for filename in filenames:
        email_analyse(os.path.join(directory, filename), to_email_list, from_email_list, email_body )

print("\nTo email adresses: \n")
print(Counter(to_email_list).most_common(10))

print("\nFrom email adresses: \n")
print(Counter(from_email_list).most_common(10))

#print(email_body)[:10]




To email adresses: 

[('richard.shapiro@enron.com', 15149), ('jeff.dasovich@enron.com', 14207), ('tana.jones@enron.com', 12828), ('steven.kean@enron.com', 12754), ('sara.shackleton@enron.com', 11433), ('james.steffes@enron.com', 10347), ('mark.taylor@enron.com', 9787), ('pete.davis@enron.com', 9281), ('susan.mara@enron.com', 9064), ('paul.kaufman@enron.com', 8522)]

From email adresses: 

[('kay.mann@enron.com', 16735), ('vince.kaminski@enron.com', 14368), ('jeff.dasovich@enron.com', 11411), ('pete.davis@enron.com', 9149), ('chris.germany@enron.com', 8801), ('sara.shackleton@enron.com', 8777), ('enron.announcements@enron.com', 8587), ('tana.jones@enron.com', 8490), ('steven.kean@enron.com', 6759), ('kate.symes@enron.com', 5438)]


# Analysis 2: Since we see that majority messages were sent to Richard Shapiro, We do sentiment analysis on the inbox messages received based on the message body.We see that majority of messages had a positive tone.

In [38]:


import os
from collections import Counter
import requests
from pprint import pprint
import argparse
import os
import json
import glob
from operator import itemgetter
import csv
from csv import DictWriter as DictWriter
from textblob import TextBlob

from email.parser import Parser
rootdir = "C:\\Users\\mahesh\\midterm\\data\\enron\\enron_mail_20150507\\maildir\\richey-c\\inbox"


#d = dict(itertools.zip_longest(*[iter(l)] * 2, fillvalue=""))

def email_analyse(inputfile, to_email_list, from_email_list, message_id_list,email_body):
    with open(inputfile, "r") as f:
        data = f.read()

    email = Parser().parsestr(data) 
    #print(email)
    
    
    
    if email['to']:
        message_id = email['Message-ID']
        email_to = email['to']
        email_to = email_to.replace("\n", "")
        email_to = email_to.replace("\t", "")
        email_to = email_to.replace(" ", "")

        email_to = email_to.split(",")

        for email_to_1 in email_to:
            to_email_list.append(email_to_1)

            from_email_list.append(email['from'])
    if email['Message-ID']:
            message_id_list.append(email['Message-ID'])
            print(email['Message-ID'])  

            email_body.append(email.get_payload())
                        
            row = str([cell.encode('utf-8') for cell in email_body])
            analysis=TextBlob(row)
            print (analysis.sentiment) 
            if analysis.sentiment.polarity < 0:
                sentiment = "negative"
            elif analysis.sentiment.polarity == 0:
                sentiment = "neutral"
            else:
                sentiment = "positive"
            print (sentiment)


to_email_list = []
from_email_list = []
email_body = []
message_id_list=[]

for directory, subdirectory, filenames in  os.walk(rootdir):
    for filename in filenames:
        email_analyse(os.path.join(directory, filename), to_email_list, from_email_list, message_id_list, email_body )

print("\nTo email adresses: \n")
print(Counter(to_email_list).most_common(15))

print("\nFrom email adresses: \n")
print(Counter(from_email_list).most_common(15))

'''
with open("to_email_list.txt", "w") as f:
    for to_email in to_email_list:
        if to_email:
            f.write(to_email)
            f.write("\n")
with open("from_email_list.txt", "w") as f:
    for from_email in from_email_list:
        if from_email:
            f.write(from_email)
            f.write("\n")        
'''   
with open("email_body.txt", "w") as f:
    for email_bod in email_body:
        if email_bod:
            f.write(email_bod)
            f.write("\n") 
            
            


<30742200.1075841468157.JavaMail.evans@thyme>
Sentiment(polarity=-0.07142857142857142, subjectivity=0.14285714285714285)
negative
<8823489.1075841468467.JavaMail.evans@thyme>
Sentiment(polarity=0.045634920634920646, subjectivity=0.44206349206349205)
positive
<3069608.1075841468615.JavaMail.evans@thyme>
Sentiment(polarity=0.10600884622623755, subjectivity=0.5021895978417718)
positive
<17749675.1075841468639.JavaMail.evans@thyme>
Sentiment(polarity=0.04259710197210198, subjectivity=0.5302454104537438)
positive
<19890272.1075841468665.JavaMail.evans@thyme>
Sentiment(polarity=0.06890998494259364, subjectivity=0.497682178932179)
positive
<26560317.1075841468688.JavaMail.evans@thyme>
Sentiment(polarity=0.059342306998557, subjectivity=0.4882549452861953)
positive
<24646221.1075841468711.JavaMail.evans@thyme>
Sentiment(polarity=0.059342306998557, subjectivity=0.4882549452861953)
positive
<33133184.1075841468738.JavaMail.evans@thyme>
Sentiment(polarity=0.05585158305746541, subjectivity=0.479632

In [40]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

with open("email_body.txt", "r") as f:
    data = f.read()


words= word_tokenize(data)

useful_words = [word  for word in words if word not in stopwords.words('English')]

frequency = nltk.FreqDist(useful_words)

print(frequency.most_common(100))

[(',', 44), ('.', 32), ('-', 26), ('I', 19), (':', 17), ('>', 14), ('@', 13), ('--', 12), ('would', 10), ('2000', 8), (';', 8), ('<', 8), ('?', 7), ('like', 7), ('July', 6), ('(', 6), (')', 6), ('Lay', 6), ('Fund', 5), ('Pate', 5), ('Zach', 5), ('Linda', 4), ('Beau', 4), ('Herrold', 4), ('4', 4), ('Howard', 4), ('birth', 4), ('address', 4), ('att1.htm', 3), ('e-mail', 3), ('time', 3), ('June', 3), ('Subject', 3), ('house', 3), ('know', 3), ('aol.com', 3), ('VERY', 3), ('Love', 3), ('David', 3), ('Message', 3), ('Family', 3), ('Sent', 3), ('If', 3), ('From', 3), ('&', 3), ('new', 3), ('Natalie', 3), ('We', 3), ('To', 3), ('PM', 3), ('Mark', 3), ('photos', 3), ('one', 2), ('back', 2), ('lplpi.com', 2), ('force', 2), ('2nd.', 2), ('stay', 2), ('asked', 2), ('beau', 2), ('Poppopindc', 2), ('[', 2), ('meet', 2), ('tell', 2), ('mailto', 2), ('Dear', 2), ('Baby', 2), ('let', 2), ('well', 2), ('return', 2), (']', 2), ('mrslinda', 2), ('send', 2), ('Big', 2), ('good', 2), ('This', 2), ('Ken', 2

In [12]:

from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS
import matplotlib.pyplot as plt
import wordcloud
import os
os.path
from wordcloud import WordCloud
from PIL import Image
#import scikits

from email.parser import Parser
rootdir = "C:\\Users\\mahesh\\midterm\\data\\enron\\enron_mail_20150507\\maildir\\lay-k\\family"

for directory, subdirectory, filenames in  os.walk(rootdir):
    for filename in filenames:
        email_analyse(os.path.join(directory, filename), to_email_list, from_email_list, email_body, email_date )


def email_analyse(inputfile, to_email_list, from_email_list, email_body, email_date):
    with open(inputfile, "r") as f:
        data = f.read()

        email = Parser().parsestr(data) 

        subjects = ' '.join(email['subject'])  
fig, ax = plt.subplots(figsize=(16, 12))
wc = wordcloud.WordCloud(width=800, 
                         height=600, 
                         max_words=200,
                         stopwords=ENGLISH_STOP_WORDS).generate(subjects)
ax.imshow(wc)
ax.axis("off") 

ImportError: No module named 'wordcloud'

In [46]:
import requests
from pprint import pprint
import argparse
import os
import json
import glob
from operator import itemgetter
import csv
from csv import DictWriter as DictWriter
from textblob import TextBlob

In [47]:
#import twython
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [15]:
#Printing messages of Richard

import os
from collections import Counter
import nltk
from nltk import tokenize
from nltk.tokenize import word_tokenize
from nltk.sentiment.vader import SentimentIntensityAnalyzer

from email.parser import Parser 
rootdir = "C:\\Users\\mahesh\\midterm\\data\\enron\\enron_mail_20150507\\maildir\\richey-c\\inbox"


def email_analyse(inufile, to_email_list, from_email_list, email_body, email_date):
    with open(inufile, "r") as f:
        data = f.read() 

    email = Parser().parsestr(data)  
    
    if email['to']: 
        email_to = email['to']
        email_to = email_to.replace("\n", "")
        email_to = email_to.replace("\t", "")
        email_to = email_to.replace(" ", "")

        email_to = email_to.split(",")

        for email_to_1 in email_to:
            to_email_list.append(email_to_1)

    from_email_list.append(email['from'])

    email_body.append(email.get_payload())
    email_date.append(email['date'])


to_email_list = []
from_email_list = []
email_body = []
email_date=[] 

for directory, subdirectory, filenames in  os.walk(rootdir):
    for filename in filenames:
        email_analyse(os.path.join(directory, filename), to_email_list, from_email_list, email_body, email_date )

#print("\nTo email adresses: \n")
#print(Counter(to_email_list).most_common(15)) 

#print("\nFrom email adresses: \n")
#print(Counter(from_email_list).most_common(15)) 


#print("\nDate: \n")
#print(Counter(email_date).most_common(15))

'''
with open("to_email_list.txt", "w") as f:
    for to_email in to_email_list:
        if to_email:
            f.write(to_email)
            f.write("\n")
with open("from_email_list.txt", "w") as f:
    for from_email in from_email_list:
        if from_email:
            f.write(from_email)
            f.write("\n")        
'''   

            
with open("email_body_richey.txt", "w") as f:
    for email_bod in email_body:
        if email_bod:
            f.write(email_bod)
            f.write("\n")
  

with open("email_body_richey.txt", "r") as f:
        data = f.read()
words= word_tokenize(data)
#lines_list = tokenize.sent_tokenize("email_body")
print(words)
            

            
            

            

['http', ':', '//story.news.yahoo.com/news', '?', 'tmpl=story', '&', 'u=/020206/170/12oyw.html', 'Jai', 'Hawker', 'Peters', '&', 'Co.', 'Limited', '(', '403', ')', '261-2259', '--', '--', '-Original', 'Message', '--', '--', '-', 'From', ':', 'Taylor', ',', 'Liz', 'Sent', ':', 'Wednesday', ',', 'February', '06', ',', '2002', '11:26', 'AM', 'To', ':', 'Lavorato', ',', 'John', ';', 'Martin', ',', 'Thomas', 'A.', ';', 'Allen', ',', 'Phillip', 'K.', ';', 'Grigsby', ',', 'Mike', ';', 'Shively', ',', 'Hunter', 'S.', ';', 'Presto', ',', 'Kevin', 'M.', ';', 'Belden', ',', 'Tim', ';', 'Swerzbin', ',', 'Mike', ';', 'Zufferli', ',', 'John', ';', 'Davis', ',', 'Mark', 'Dana', 'Cc', ':', 'Kitchen', ',', 'Louise', ';', 'Hillis', ',', 'Kimberly', ';', 'Vuittonet', ',', 'Laura', ';', 'Rangel', ',', 'Ina', ';', 'Hogan', ',', 'Irena', 'D.', ';', 'Black', ',', 'Tamara', 'Jae', 'Subject', ':', 'Trader', 'Meeting', '-', '2/7', '@', '7:45', 'a.m.', '(', 'CST', ')', '-', 'ECS', '06752', 'Greg', 'Whalley', 'ha

# Analysis 3: Calculating the peak time of communication in a day for Richey. We see that maximum communications were exchanged at 2pm.
#Here 18 denotes 6pm and so on(24 hour clock)

In [26]:
#Time Analysis

import os
import operator
from email.parser import Parser
rootdir = "C:\\Users\\mahesh\\midterm\\data\\enron\\enron_mail_20150507\\maildir\\richey-c\\inbox"


def email_analyse(inputfile, to_email_list, from_email_list, email_body, email_time):
    with open(inputfile, "r") as f:
        data = f.read()

    email = Parser().parsestr(data)

    to_email_list.append(email['to'])
    from_email_list.append(email['from'])

    email_body.append(email.get_payload())
    email_time.append(email.get('date'))


to_email_list = []
from_email_list = []
email_body = []
email_time=[]
t=[]
time=[]
time_dic={}

for directory, subdirectory, filenames in  os.walk(rootdir):
    for filename in filenames:
        email_analyse(os.path.join(directory, filename), to_email_list, from_email_list, email_body, email_time )
            
with open("email_time.txt","w") as f:
    for email_tim in email_time:
        if email_tim:
            email_tim=email_tim.split(" ")
            for time_1 in email_tim[4:5]:
                time_1=time_1.split(":")
                t.append(time_1)
                for t1 in time_1[0:1]:
                    #print(time_1)
                    time.append(t1)

for digit in time:
    if digit in time_dic:
        time_dic[digit] +=1
    else:
        time_dic[digit]=1
#print (time_dic)

sorted_x = sorted(time_dic.items(), key=operator.itemgetter(1))
print(sorted_x)

[('18', 1), ('23', 1), ('22', 1), ('05', 2), ('06', 2), ('19', 2), ('10', 4), ('07', 4), ('17', 4), ('16', 4), ('08', 5), ('15', 5), ('11', 5), ('09', 6), ('13', 6), ('12', 7), ('14', 9)]


In [2]:
import sys
print(sys.path)

['', 'C:\\Python27', 'C:\\Python27\\Scripts', 'C:\\Users\\mahesh\\Desktop\\Data analysis spring 2017\\Data_Analysis_Assignments\\wagh_rijuta_spring2017\\Midterm', 'C:\\Users\\mahesh\\Miniconda3\\python35.zip', 'C:\\Users\\mahesh\\Miniconda3\\DLLs', 'C:\\Users\\mahesh\\Miniconda3\\lib', 'C:\\Users\\mahesh\\Miniconda3', 'C:\\Users\\mahesh\\Miniconda3\\lib\\site-packages', 'C:\\Users\\mahesh\\Miniconda3\\lib\\site-packages\\win32', 'C:\\Users\\mahesh\\Miniconda3\\lib\\site-packages\\win32\\lib', 'C:\\Users\\mahesh\\Miniconda3\\lib\\site-packages\\Pythonwin', 'C:\\Users\\mahesh\\Miniconda3\\lib\\site-packages\\setuptools-27.2.0-py3.5.egg', 'C:\\Users\\mahesh\\Miniconda3\\lib\\site-packages\\IPython\\extensions', 'C:\\Users\\mahesh\\.ipython']


In [3]:
import sys
sys.path.append('C:\python34\lib\site-packages')

In [9]:
import textblob

In [44]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from os import path
from scipy.misc import imread
import matplotlib.pyplot as plt
import random

from wordcloud import WordCloud, STOPWORDS

with open("email_body.txt", "r") as f:
    data = f.read()


words= word_tokenize(data)
print(words)

wordcloud = WordCloud(relative_scaling = 1.0,
                      stopwords = 'to of : ; -- and the').generate(words)

ImportError: Module use of python34.dll conflicts with this version of Python.

In [42]:
import sys
print(sys.path)

['', 'C:\\Python27', 'C:\\Python27\\Scripts', 'C:\\Users\\mahesh\\Desktop\\Data analysis spring 2017\\Data_Analysis_Assignments\\wagh_rijuta_spring2017\\Midterm', 'C:\\Users\\mahesh\\Miniconda3\\python35.zip', 'C:\\Users\\mahesh\\Miniconda3\\DLLs', 'C:\\Users\\mahesh\\Miniconda3\\lib', 'C:\\Users\\mahesh\\Miniconda3', 'C:\\Users\\mahesh\\Miniconda3\\lib\\site-packages', 'C:\\Users\\mahesh\\Miniconda3\\lib\\site-packages\\win32', 'C:\\Users\\mahesh\\Miniconda3\\lib\\site-packages\\win32\\lib', 'C:\\Users\\mahesh\\Miniconda3\\lib\\site-packages\\Pythonwin', 'C:\\Users\\mahesh\\Miniconda3\\lib\\site-packages\\setuptools-27.2.0-py3.5.egg', 'C:\\Users\\mahesh\\Miniconda3\\lib\\site-packages\\IPython\\extensions', 'C:\\Users\\mahesh\\.ipython', 'C:\\python34\\lib\\site-packages']


In [43]:
sys.path.append('C:\python34\lib\site-packages')